# Строгая экзогенность
<br>
- **Вопрос 1:** если в целевой функции две зависимых переменных, а в модели построена только одна, то насколько конкретно будет смещена оценка коэффициента перед переменной в модели? Рассмотрим два случая: когда имеется confounding-переменная (т.е. истинная причина, которая влияет на зависимую и независимую переменные), и когда имеется просто недоступная нам переменная, которая не влияет на имеющуюся у нас переменную, однако при этом её матожидание не равно нулю, а оттуда и матожидание шума получится ненулевое, то есть предположение о строгой экзогенности будет нарушено.

- **Вопрос 2:** если в целевой функции $\text{y}$ зависит от $\text{X и X^2}$, а в модели - только от $\text{X}$ (то есть $\text{X^2}$ окажется в шуме), то насколько конкретно будет смещена оценка коэффициента перед переменной в модели?

<br><br><br><hr>
## 1. Наблюдение первое - confounding variable и дополнительная переменная
Рассмотрим следующий случай. Пусть на основе имеющихся данных мы регрессируем Зарплату на Образование, и построенная нами модель имеет вид:

$$\text{Wages} = \beta_0 + \beta_1\cdot\text{Education} + \varepsilon$$

Однако в реальности отношение несколько иное:

$$\text{Wages} = \beta_0 + \beta_1\cdot \text{Education} + \beta_2\cdot\text{Ability} + \mu$$

Способности в данном случае влияют на Образование и на Зарплату: чем выше Способности - тем больше Образование (более способные ученики чаще получают лучшее образование) и тем больше прогнозируемая Зарплата (более способные люди чаще зарабатывают больше). Между тем, Образование само по себе не очень сильно влияет на Зарплату: можно учиться долго и много, но зарабатывать мало, тогда как несколько менее образованный но более "пробивной" работник может подняться выше по карьерной лестнице. Легко показать, что есть матожидание Способностей ненулевое, то $\mathsf{E}[\varepsilon]\neq 0$, то есть нарушается экзогенность. Рассмотрим эту ситуацию подробнее и обратим внимание на оценки коэффициентов Образования.

<br><br><center>**Опыт 1. Confounding-переменная, влияет на X и y**</center>

Введём следующие условия:

**Условия:**
1. Зависимая переменная регрессируется на одну независимую переменную.
2. Присутствует скрытая переменная, влияющая как на х, так и на у.

**Объект интереса:**
- Оценка коэффициента $\text{x}$ и её смещение - endgogeneity bias 

**Ожидание:**

- ///

In [234]:
# General:
import numpy as np
# Graphics:
import seaborn as sns
from matplotlib import pyplot as plt
# ML:
import statsmodels.api as sm
from sklearn.datasets import make_regression
FONT_BOLD, FONT_END = '\033[1m', '\033[0m'

BIAS  = 4.5
NOISE = 12
POPULATION_SIZE = 500
SAMPLE_SIZE     = int(POPULATION_SIZE/5)


# 1. Сгенерируем проблему регрессии с двумя независимыми переменными:
X, y, real_coef = make_regression(n_samples=POPULATION_SIZE, n_features=2, n_informative=1, 
                                  coef=True, noise=NOISE, bias=BIAS)
print(FONT_BOLD, ' -- Сгенерирована целевая функция.\n', FONT_END, '    Значения коэффициентов целевой функции:')
print('\t· β0 =', BIAS, '\n\t· β1 =', real_coef.item(0), '\n\t· β2 =', real_coef.item(1))
informative_feature_index = int(real_coef.item(0) == 0)
informative_feature_coef  = max(real_coef.item(0), real_coef.item(1))

# 2. Запишем коэффициент корреляции Пирсона между информативным и неинформативным признаками:
inter_coef          = np.corrcoef([row[0] for row in X], [row[1] for row in X])[0, 1]
informative_coef    = np.corrcoef([row[informative_feature_index] for row in X], y)[0, 1]
noninformative_coef = np.corrcoef([row[1 - informative_feature_index] for row in X], y)[0, 1]
print('    Корреляция между информативным и неинформативным признаками:', inter_coef)
print('    Корреляция между информативным признаком и y:', ' '*14, informative_coef)
print('    Корреляция между неинформативным признаком и y:', ' '*12, noninformative_coef)

# 3. Экстрагируем информативный признак, и регрессируем y на неинформативный признак:
new_X = np.delete(X, informative_feature_index, 1)
statsmodels_result = sm.OLS(y, sm.add_constant(new_X)).fit()
print(FONT_BOLD, '\n\n -- Построена модель (на основании неинформативного признака).', FONT_END)
print('    Оценки коэффициентов регрессии:\n\t· Оценка β0:', statsmodels_result.params[0], 
      '\n\t· Оценка β1:', statsmodels_result.params[1])

  -- Сгенерирована целевая функция.
      Значения коэффициентов целевой функции:
	· β0 = 4.5 
	· β1 = 0.0 
	· β2 = 21.553976954923993
    Корреляция между информативным и неинформативным признаками: -0.0373502502487
    Корреляция между информативным признаком и y:                0.862109803766
    Корреляция между неинформативным признаком и y:              -0.073345030119
 

 -- Построена модель (на основании неинформативного признака). 
    Оценки коэффициентов регрессии:
	· Оценка β0: 4.53800673976 
	· Оценка β1: -1.76683487328


In [235]:
z_X = np.delete(X, informative_feature_index, 1)
statsmodels_result = sm.OLS(y, sm.add_constant(z_X)).fit()
print('Effect of NON-INFORMATIVE on y:\n\tWith bias:', statsmodels_result.params[1])
statsmodels_result = sm.OLS(y, z_X).fit()
print('\tNo bias:  ', statsmodels_result.params[0])

z_X = np.delete(X, 1-informative_feature_index, 1)
statsmodels_result = sm.OLS(y, sm.add_constant(z_X)).fit()
print('\nEffect of INFORMATIVE on y:\n\tWith bias:', statsmodels_result.params[1])
statsmodels_result = sm.OLS(y, z_X).fit()
print('\tNo bias:  ', statsmodels_result.params[0])


z_X_1, z_X_2 = X[informative_feature_index], X[1 - informative_feature_index]
statsmodels_result = sm.OLS(z_X_2, sm.add_constant(z_X_1)).fit()
print('\nEffect of NON-INFORMATIVE on INFORMATIVE:\n\tWith bias:', statsmodels_result.params[1])
statsmodels_result = sm.OLS(z_X_2, z_X_1).fit()
print('\tNo bias:  ', statsmodels_result.params[0])

Effect of NON-INFORMATIVE on y:
	With bias: -1.76683487328
	No bias:   -1.44801061253

Effect of INFORMATIVE on y:
	With bias: 20.9079770399
	No bias:   20.9251408303

Effect of INFORMATIVE on NON-INFORMATIVE:
	With bias: -2.35648757822
	No bias:   -2.5007427365


the estimated coefficient b would pick up the effect of x on y plus the association of x and z times the effect of z on y.

In [237]:
noninformative_coef + (inter_coef * informative_coef)

-0.10554504703150308

In [239]:
-1.76683487328 + (-0.073345030119 * 20.9079770399)

-3.300331078998826

In [216]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(a)
a[:, 2]

[[1 2 3]
 [4 5 6]
 [7 8 9]]


array([3, 6, 9])

**Результат:**
* MSE на тренировочной выборке ожидаемо получилась практически нулевой, в силу того, что подгонка полинома второй степени к двум точкам всегда будет идеальной.
* Однако на популяции MSE ожидаемо оказалась выше, с мощной дисперсией. Это обусловлено тем, что через две точки можно провести бесконечное количество парабол, и какая из них будет ближе к целевой функции - абсолютно неизвестно.

Теперь проведём такой же опыт, но уберём вариативность, задав как минимум 3 точки для тренировочного набора. Для точного задания параболы необходимо иметь как минимум 3 точки, через которые она проходит; а в силу того, что данные не имеют шума, эти 3 точки всегда будут лежать ровно на целевой параболе. Таким образом, ожидается, что подгонка всегда будет идеальной.

<br><br><center>**Опыт 2. Нулевой шум, нулевая вариативность**</center>

**Условия:**
1. Стохастический шум равен нулю: $\forall{x}, \epsilon(x)=0$
2. Детерминистский шум равен нулю: $\mathrm{Complexity}(g)=\mathrm{Complexity}(f)$
3. Вариативность модели нулевая; в данном случае (т.е. в случае с целевой функцией-параболой) объём выборки должен быть $\geq 3$.

**Ожидание:**

- Oжидается нулевая MSE как на тренировке, так и на популяции. Это будет достигнуто за счёт нулевой вариативности, нулевого детерминистского и нулевого стохастического шума.